
# Intro to LangChain

![](https://python.langchain.com/svg/langchain_stack_112024_dark.svg)

LangChain is an open-source framework that gives developers the tools they need to create applications using large language models (LLMs). In its essence, LangChain is a prompt orchestration tool that makes it easier for teams to connect various prompts interactively.

In [19]:
%pip install -qU openai
%pip install -qU langchain
%pip install -qU langchain-core
%pip install -qU langchain-experimental
%pip install -qU langchain-community
%pip install -qU langchain-openai
%pip install -qU langchain-ollama
%pip install -qU tiktoken
%pip install -qU pydantic[email]
%pip install -qU jupyter ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Accessing OpenAI Directly

In [2]:
import os
from dotenv import load_dotenv
import openai
from google import genai

load_dotenv("../../.env", override=True)

True

In [4]:
#os.environ["OPENAI_API_KEY"] = "<the key>"
#openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()
model="gpt-5-nano"

In [7]:
def get_completion(prompt, model=model):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
        ]
    response = client.chat.completions.create(model=model,
                                              messages=messages,
                                              )
    return response.choices[0].message.content

## Simple Query

In [8]:
response = get_completion("Can you tell me how much is 1+1?")
print(response)

2. In standard decimal arithmetic, 1 + 1 = 2. In base-2 (binary), it's written as 10.


## Queries with custom prompts using formatted strings

In [9]:
customer_email = """
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls
with smoothie! And to make matters worse, the warranty don't cover the cost of
cleaning up me kitchen. I need yer help right now, matey!
"""

style = """American English in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls
with smoothie! And to make matters worse, the warranty don't cover the cost of
cleaning up me kitchen. I need yer help right now, matey!
```



In [10]:
response = get_completion(prompt)
print(response)

I'm upset that my blender lid flew off and splattered my kitchen walls with smoothie. To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now.


## Why should we use a framework to wrap the direct calls to OpenAI?    

The LangChain pipelines consist of the following modules:

+ **Models**: Models mostly cover Large Language models. A large language model of considerable size is a model that comprises a neural network with numerous parameters and is trained on vast quantities of unlabeled text  
+ **Prompts**: prompt is the input that we give to any system to refine our answers to make them more accurate or more specific according to our use case. Many times you may want to get more structured information than just text back. We can use Prompts in conjunction with Parsers
+ **Parsers**: Output parsers are responsible for taking the output of an LLM and transforming it to a more suitable format. This is very useful when you are using LLMs to generate any form of structured data.  
+ **Memory**: Chains and Agents in LangChain operate in a stateless mode by default, meaning that they handle each incoming query independently. However, there are certain applications, like chatbots, where it is of great importance to retain previous interactions, both over the short and long term. This is where the concept of “Memory” comes into play.  
+ **Chains**: Chains provide a means to merge various components into a unified application. A chain can be created, for instance, that receives input from a user, formats it using a PromptTemplate, and subsequently transmits the formatted reply to an LLM. More intricate chains can be generated by integrating multiple chains with other components.  
+ **Agents**: Certain applications may necessitate not only a pre-determined sequence of LLM/other tool calls but also an uncertain sequence that is dependent on the user’s input. These kinds of sequences include an “agent” that has access to a range of tools. Based on the user input, the agent may determine which of these tools, if any, should be called.  
+ **Callbacks**: Allow you to hook into the various stages of your LLM application. This is useful for logging, monitoring, streaming, and other tasks.  
+ **Indexes**: Indexes are information stored in local databases that can be used for Augment the capabilities of the models being used. We will see them in action with RAG pipelines  

Let's wrap the pipeline:

![](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*05zEoeNU7DVYOFzjugiF_w.jpeg)


# I - Simple Pipelines with Langchain  

In [36]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import ChatMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder

from langchain_core.messages import SystemMessage
from langchain_core.messages import AIMessage
from langchain_core.messages import HumanMessage

import json
from pydantic import BaseModel, Field, model_validator, EmailStr, ValidationError
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import PydanticOutputParser


In [38]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_ollama.llms import OllamaLLM

model1 = ChatOpenAI(model="gpt-5-nano")
model2 = ChatOpenAI(model="gpt-4o")
model3 = ChatOpenAI(model="gpt-4o-mini", temperature=0)
model4 = OllamaLLM(model="gemma2:2b")

### Once you've installed and initialized the LLM of your choice, we could use it!   
### Let's ask it some question:

In [39]:
model3.invoke("Why is the sky blue?")

AIMessage(content="The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it is made up of different colors, each with varying wavelengths. Blue light has a shorter wavelength compared to other colors like red or yellow.\n\nAs sunlight passes through the atmosphere, it collides with gas molecules and small particles. Because blue light is scattered in all directions more than other colors due to its shorter wavelength, we see a predominance of blue when we look up at the sky.\n\nDuring sunrise and sunset, the sun's light has to pass through a thicker layer of the atmosphere, which scatters the shorter blue wavelengths out of our line of sight and allows the longer wavelengths, like red and orange, to dominate, giving the sky those warm colors.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 153, 'prompt_tokens': 13, 'total_tokens': 166, 'completion_tokens_details': {'acce

In [41]:
model1.invoke("Why is the sky blue?")

AIMessage(content='Because sunlight (which is white) scatters as it passes through Earth’s atmosphere. The air is full of tiny gas molecules that scatter shorter wavelengths of light (blue and violet) more than longer wavelengths (red, yellow).\n\nA few details:\n- Violet light is scattered even more than blue, but we don’t see violet in the sky because the sun’s violet is weak and the atmosphere absorbs much of it, and our eyes are less sensitive to violet.\n- The blue light that is scattered in all directions reaches our eyes from every part of the sky, so the sky looks blue most of the day.\n- At sunrise and sunset, sunlight travels through more atmosphere, scattering away the blues and leaving reds/oranges, which is why the sky can look orange or pink then.\n\nSo, the sky is blue mainly due to Rayleigh scattering of sunlight by the atmosphere.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1015, 'prompt_tokens': 12, 'total_tokens': 10

### Prompt templates convert raw user input to better input to the LLM.

In [42]:
prompt = ChatPromptTemplate.from_messages([("system", "You are a first grade teacher."),
                                           ("user", "{input}")
                                         ])

chain = prompt | model1
chain.invoke({"input": "Why is the sky blue?"})

AIMessage(content='Great question! Sunlight looks white, but it’s really made of many colors. When the sunlight hits the air in our sky, it bumps into lots of tiny air molecules. Blue light waves get scattered in all directions more than the other colors. So we see blue light coming from everywhere, and the sky looks blue most of the time.\n\nSometimes at sunrise or sunset the light has to travel through more air, so the blue is scattered away and we see red, orange, and pink colors instead. On cloudy days the sky can look gray. Want to try a quick color activity at home later?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1282, 'prompt_tokens': 23, 'total_tokens': 1305, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1152, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-

### The output of a ChatModel (and therefore, of this chain) is a message object. However, it's often much more convenient to work with strings.  
### Let's add a simple output parser to convert the chat message to a string.

In [43]:
output_parser = StrOutputParser()

In [44]:
chain = prompt | model1 | output_parser
chain.invoke({"input": "Why is the sky blue?"})

'Great question! Here’s a simple way to think about it:\n\n- Sunlight looks white, but it’s really made of many colors.\n- As the sunlight travels through the air, the tiny particles in the air scatter blue light in all directions.\n- So we see the sky as blue everywhere we look most of the day.\n- At sunrise and sunset, the light travels through more air, so the blue light gets scattered away and we see reds and oranges.\n\nA quick safety note: don’t look directly at the Sun.\n\nIf you want, we can do a tiny kid-friendly demo in class to show how light and colors work. Can you tell me what color you see in the sky today?'

# II - Exploring Chain Elements - Langchain Expression Language

Notice this line of the code, where we piece together these different components into a single chain using LCEL:  

**chain = prompt | model | output_parser**

## 1. [Prompts](https://python.langchain.com/docs/modules/model_io/prompts/)    
prompt is a BasePromptTemplate, which means it takes in a dictionary of template variables and produces a PromptValue.  
+ A PromptValue is a wrapper around a completed prompt that can be passed to either a LLM (which takes a string as input) or ChatModel (which takes a sequence of messages as input).  
+ It can work with either language model type because it defines logic both for producing BaseMessages and for producing a string.

In [45]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

chain = prompt | model1 | output_parser

chain.invoke({"topic": "chicken"})

'Why did the chicken join a band? Because it had the drumsticks. Want another one?'

In [46]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream', additional_kwargs={}, response_metadata={})])

In [47]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream', additional_kwargs={}, response_metadata={})]

In [48]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

In [49]:
prompt = ChatPromptTemplate.from_messages([("system", "You are a first grade teacher."),
                                           ("user", "{input}")
                                         ])
chain = prompt | model1 | output_parser
chain.invoke({"input": "Why is the sky blue?"})

'Great question! Here’s a simple way to think about it:\n\n- The sun shines white light, which is made of all the colors.\n- The air in our sky has tiny particles. When sunlight hits them, the light gets bounced around in many directions. This is called scattering.\n- Blue light waves are shorter, so they scatter the most. That blue light goes everywhere in the sky, so the sky looks blue most of the time.\n- At sunrise or sunset, the sun’s light travels through more air. Lots of blue light gets scattered away, and we see warm colors like red and orange.\n\nSo, blue sky because blue light scatters the most in our air. Want to draw a blue sky or try a simple color-scattering activity?'

## 2. [Models](https://python.langchain.com/docs/modules/model_io/chat/quick_start/)    
#### The PromptValue is then passed to model.  
#### In this case our model is an OpenAI ChatModel, meaning it will output a BaseMessage.

In [50]:
message = model3.invoke(prompt_value)
print(message)
print(type(message))

content='Why did the ice cream cone break up with the sundae? \n\nBecause it found someone cooler!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 15, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CyQmXPMBPa4cE607JkFKGXaXiVKuW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019bc3ed-43bf-7253-be28-76a14d93ffbb-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 15, 'output_tokens': 20, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
<class 'langchain_core.messages.ai.AIMessage'>


#### If our model was an Ollama LLM, it would output a string.

In [38]:
message = model4.invoke(prompt_value)
print(message)
print(type(message))

Why did the ice cream cone go to the doctor? 

Because it was feeling crumby! 😂🍦 

<class 'str'>


#### That is why we should treat the model output with an output parser

## 3. [Output parsers](https://python.langchain.com/docs/modules/model_io/output_parsers/)    
And lastly we pass our model output to the output_parser, which is a BaseOutputParser meaning it takes either a string or a BaseMessage as input. The specific StrOutputParser simply converts any input into a string.

In [51]:
output_parser.invoke(message)

'Why did the ice cream cone break up with the sundae? \n\nBecause it found someone cooler!'

In [52]:
print(type(message))

<class 'langchain_core.messages.ai.AIMessage'>


## 4. [Chains](https://python.langchain.com/v0.1/docs/modules/chains/)  

#### Chains are combination of steps. We have followed the steps along:

+ We passed the user input on the desired topic as {"topic": "ice cream"}
+ The prompt component takes the user input, which is then used to construct a PromptValue after using the topic to construct the prompt.
+ The model component takes the generated prompt, and passes into the OpenAI LLM model for evaluation.
+ The generated output from the model is a ChatMessage object.
+ Finally, the output_parser component takes in a ChatMessage, and transforms this into a Python string, which is returned from the invoke method.

#### Langchain has many types of chains using [LCEL](https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel). We are just exploring some in this notebook. 

# 5. [Types of Prompts](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/)  
#### Let's explore some other types of Prompts  

## 5.1 PromptTemplate

Use PromptTemplate to create a template for a string prompt.  
By default, PromptTemplate uses Python’s str.format syntax for templating.

In [53]:
prompt_template = PromptTemplate.from_template("Tell me a {adjective} joke about {content}.")
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [54]:
prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

## 5.2 ChatPromptTemplate

The prompt to chat models/ is a list of chat messages.  
Each chat message is associated with content, and an additional parameter called role. For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.

In [55]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
messages

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]

#### This is equivalent as doing the following, directly with OpenAI

In [56]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful AI bot. Your name is Bob."},
        {"role": "user", "content": "Hello, how are you doing?"},
        {"role": "assistant", "content": "I'm doing well, thanks!"},
        {"role": "user", "content": "What is your name?"},
    ],
)
response

ChatCompletion(id='chatcmpl-CyQn4YiisuIMeXurWu1NRQvxLlEiF', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='My name is Bob. How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1768518698, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_29330a9688', usage=CompletionUsage(completion_tokens=12, prompt_tokens=49, total_tokens=61, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

The ChatPromptTemplate.from_messages static method accepts a variety of message representations and is a convenient way to format input to chat models with exactly the messages you want.  

For example, in addition to using the 2-tuple representation of (type, content) used above, you could pass in an instance of MessagePromptTemplate or BaseMessage.

In [57]:
chat_template = ChatPromptTemplate.from_messages(
    [SystemMessage(content=("You are a helpful assistant that re-writes the user's text to "
                            "sound more upbeat.")),
     HumanMessagePromptTemplate.from_template("{text}"),
    ])


messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat.", additional_kwargs={}, response_metadata={}), HumanMessage(content="I don't like eating tasty things", additional_kwargs={}, response_metadata={})]


## 5.3 Message Prompts  
LangChain provides different types of MessagePromptTemplate. The most commonly used are
+ AIMessagePromptTemplate  
+ SystemMessagePromptTemplate  
+ HumanMessagePromptTemplate  

Which create an AI message, system message and human message respectively.  
In cases where the chat model supports taking chat message with arbitrary role, you can use ChatMessagePromptTemplate, which allows user to specify the role name.  
https://python.langchain.com/docs/modules/model_io/chat/message_types/  

In [58]:
prompt = "May the {subject} be with you"

chat_message_prompt = ChatMessagePromptTemplate.from_template(role="Jedi", template=prompt)
chat_message_prompt.format(subject="force")

ChatMessage(content='May the force be with you', additional_kwargs={}, response_metadata={}, role='Jedi')

# 5.4 MessagesPlaceholder  
LangChain also provides MessagesPlaceholder, which gives you full control of what messages to be rendered during formatting. This can be useful when you are uncertain of what role you should be using for your message prompt templates or when you wish to insert a list of messages during formatting.

In [59]:
human_prompt = "Summarize our conversation so far in {word_count} words."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)
chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder(variable_name="conversation"),
     human_message_template]
)

human_message = HumanMessage(content="What is the best way to learn programming?")
ai_message = AIMessage(content="""\
1. Choose a programming language: Decide on a programming language that you want to learn.
2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, 
data types and control structures.
3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
"""
)

chat_prompt.format_prompt(conversation=[human_message, ai_message], word_count="10").to_messages()

[HumanMessage(content='What is the best way to learn programming?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='1. Choose a programming language: Decide on a programming language that you want to learn.\n2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, \ndata types and control structures.\n3. Practice, practice, practice: The best way to learn programming is through hands-on experience', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='Summarize our conversation so far in 10 words.', additional_kwargs={}, response_metadata={})]

# 6 - [Types of Output Parsers](https://python.langchain.com/docs/modules/model_io/output_parsers/)  

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in. Output parsers are classes that help structure language model responses.  
There are two main methods an output parser must implement:  
+ “Get format instructions”: A method which returns a string containing instructions for how the output of a language model should be formatted.
+ “Parse”: A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

And then one optional one:

+ “Parse with prompt”: A method which takes in a string (assumed to be the response from a language model) and a prompt (assumed to be the prompt that generated such a response) and parses it into some structure. The prompt is largely provided in the event the OutputParser wants to retry or fix the output in some way, and needs information from the prompt to do so.

#### Let's explore the **CSV Parser**

In [61]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(template="List five {subject}.\n{format_instructions}",
                        input_variables=["subject"],
                        partial_variables={"format_instructions": format_instructions},
)

chain = prompt | model1 | output_parser
a = chain.invoke({"subject": "ice cream flavors"})

In [62]:
print(a)
print(type(a))

['vanilla', 'chocolate', 'strawberry', 'mint chocolate chip', 'cookies and cream']
<class 'list'>


In [63]:
for s in chain.stream({"subject": "ice cream flavors"}):
    print(s)

['vanilla']
['chocolate']
['strawberry']
['mint chocolate chip']
['cookies and cream']


#### Below we go over a more powerful output parser, the **PydanticOutputParser**.

#### a) Define your desired data structure.

In [64]:
class UserInfo(BaseModel):
    name: str
    age: int
    email: EmailStr

#### b) Create a parser instance for the schema

In [65]:
parser = PydanticOutputParser(pydantic_object=UserInfo)

#### c) Create a prompt that formats the output in the required structure

In [66]:
prompt_template = """
Extract the following information from the text:

name: The full name of the user.
age: The user's age (as an integer).
email: The user's email address.

Return the data in JSON format. 

Here's the text:
{text}
"""

#### d) Set up the Prompt Template

In [67]:
prompt = PromptTemplate(
    input_variables=["text"],
    template=prompt_template
)

# Example text input
text_input = "John Doe, a 30-year-old, can be reached at john.doe@example.com."

#### e) And a query intended to prompt a language model to populate the data structure.

In [68]:
prompt_and_model = prompt | model3
output = prompt_and_model.invoke({"text": text_input})
print(output.content)

```json
{
  "name": "John Doe",
  "age": 30,
  "email": "john.doe@example.com"
}
```


In [69]:
result = parser.invoke(output)
result

UserInfo(name='John Doe', age=30, email='john.doe@example.com')

In [71]:
result.model_dump_json()

'{"name":"John Doe","age":30,"email":"john.doe@example.com"}'

### A more complex example

#### a) Define your desired data structure.

In [72]:
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup")
        if setup and setup[-1] != "?":
            raise ValueError("Badly formed question!")
        return values

#### b) Set up a parser and the prompt template.

In [73]:
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

#### c) And a query intended to prompt a language model to populate the data structure.

In [74]:
prompt_and_model = prompt | model1
output = prompt_and_model.invoke({"query": "Tell me a joke."})

result = parser.invoke(output)
result

Joke(setup='Why did the scarecrow win an award?', punchline='Because he was outstanding in his field.')

In [75]:
result.model_dump_json()

'{"setup":"Why did the scarecrow win an award?","punchline":"Because he was outstanding in his field."}'

# 7 - [Memory](https://python.langchain.com/v0.1/docs/modules/memory/)  

Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation. At bare minimum, a conversational system should be able to access some window of past messages directly. A more complex system will need to have a world model that it is constantly updating, which allows it to do things like maintain information about entities and their relationships.

TBD - Completely revamped in Langchain